In [2]:
# script_bdpm_nom_dci_atc.py
# dépendances : pandas, requests
# python >= 3.8 recommandé

import os
import requests
import pandas as pd

# ========== paramètres ==========
out_dir = "data/bdpm_files"
os.makedirs(out_dir, exist_ok=True)

# URLs officielles (page téléch. BDPM)
urls = {
    "CIS_bdpm.txt": "https://base-donnees-publique.medicaments.gouv.fr/index.php/download/file/CIS_bdpm.txt",
    "CIS_COMPO_bdpm.txt": "https://base-donnees-publique.medicaments.gouv.fr/index.php/download/file/CIS_COMPO_bdpm.txt",
    "CIS_MITM.txt": "https://base-donnees-publique.medicaments.gouv.fr/index.php/download/file/CIS_MITM.txt"
}

In [3]:
# ========== téléch. si nécessaire ==========
for name, url in urls.items():
    path = os.path.join(out_dir, name)
    if not os.path.exists(path):
        print(f"Téléchargement de {name} ...")
        r = requests.get(url, timeout=60)
        r.raise_for_status()
        with open(path, "wb") as f:
            f.write(r.content)
        print("   OK:", path)
    else:
        print("Déjà présent :", path)


Déjà présent : data/bdpm_files\CIS_bdpm.txt
Déjà présent : data/bdpm_files\CIS_COMPO_bdpm.txt
Déjà présent : data/bdpm_files\CIS_MITM.txt


In [4]:

# ========== lecture des fichiers ==========
# Rappel : fichiers tabulés, sans en-tête
cis_path = os.path.join(out_dir, "CIS_bdpm.txt")
compo_path = os.path.join(out_dir, "CIS_COMPO_bdpm.txt")
mitm_path = os.path.join(out_dir, "CIS_MITM.txt")


In [5]:
# Lecture brute (tout en str pour éviter problèmes de parsing)


cis = pd.read_csv(cis_path, sep="\t", header=None, dtype=str, encoding="latin-1", low_memory=False)
compo = pd.read_csv(compo_path, sep="\t", header=None, dtype=str, encoding="latin-1", low_memory=False)
mitm = pd.read_csv(mitm_path, sep="\t", header=None, dtype=str, encoding="latin-1", low_memory=False)



In [6]:
# ========== colonne indices (conformément au PDF officiel) ==========
# CIS_bdpm.txt : 0 = Code CIS, 1 = Dénomination du médicament (nom commercial)
# CIS_COMPO_bdpm.txt : 0 = Code CIS, 3 = Dénomination de la substance, 6 = Nature du composant (SA/ST)
# CIS_MITM.txt : 0 = Code CIS, 1 = Code ATC

# Renommer colonnes utiles
cis = cis[[0, 1]].copy()
cis.columns = ["CIS", "nom_commercial"]

# Certaines lignes peuvent contenir espaces; on nettoie
cis["CIS"] = cis["CIS"].str.strip()
cis["nom_commercial"] = cis["nom_commercial"].str.strip()

In [8]:

# Composition : on garde les colonnes 0 (CIS), 3 (dénomination substance), 6 (nature)
# ATTENTION : si le format change, adaptez les indices.
compo_small = compo[[0, 3, 6]].copy()
compo_small.columns = ["CIS", "dci", "nature"]
compo_small["CIS"] = compo_small["CIS"].str.strip()
compo_small["dci"] = compo_small["dci"].str.strip().fillna("")
compo_small["nature"] = compo_small["nature"].str.strip().fillna("")

In [9]:
# Filtrer sur nature == 'SA' (substance active)
compo_SA = compo_small[compo_small["nature"].str.upper() == "SA"].copy()


In [10]:
# Agréger les DCI par CIS (uniques et concaténés par ;)
compo_grouped = (compo_SA.groupby("CIS")["dci"]
                       .apply(lambda s: "; ".join(sorted(set([x for x in s if x and pd.notna(x)]))))
                       .reset_index()
                       .rename(columns={"dci": "dci_aggregated"}))

In [ ]:
# vih = df_grouped[df_grouped["atc"].str.startswith("J05", na=False)]

In [11]:
# MITM : CIS -> ATC (col 0 CIS, col 1 ATC)
mitm_small = mitm[[0, 1]].copy()
mitm_small.columns = ["CIS", "atc"]
mitm_small["CIS"] = mitm_small["CIS"].str.strip()
mitm_small["atc"] = mitm_small["atc"].str.strip().fillna("")


In [12]:

# ========== fusion finale ==========
# fusionner nom commercial (cis) + dci + atc
df = cis.merge(compo_grouped, on="CIS", how="left")
df = df.merge(mitm_small, on="CIS", how="left")

In [13]:
# renommons colonnes finales et reordonnons
df_final = df[["CIS", "nom_commercial", "dci_aggregated", "atc"]].copy()
df_final.columns = ["cis", "nom_commercial", "dci", "atc"]

In [14]:
# Remplacer NaN par chaîne vide
df_final["dci"] = df_final["dci"].fillna("")
df_final["atc"] = df_final["atc"].fillna("")


In [17]:
# ========== sortie ==========
df_final

,cis,nom_commercial,dci,atc
0,61266250,"A 313 200 000 UI POUR CENT, pommade","CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",
1,62869109,"A 313 50 000 U.I., capsule molle","CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",
2,69103878,"A.D.N. BOIRON, degré de dilution compris entre...",A.D.N. POUR PRÉPARATIONS HOMÉOPATHIQUES,
3,61876780,"ABACAVIR ARROW 300 mg, comprimé pelliculé sécable",SULFATE D'ABACAVIR,J05AF06
4,63797011,"ABACAVIR SANDOZ 300 mg, comprimé pelliculé séc...",ABACAVIR,J05AF06
...,...,...,...,...
15818,64949486,"ZYRTECSET 10 mg, comprimé pelliculé sécable",CÉTIRIZINE (DICHLORHYDRATE DE),
15819,67337081,"ZYTIGA 500 mg, comprimé pelliculé",ACÉTATE D'ABIRATÉRONE,L02BX03
15820,63095061,"ZYVOXID 100 mg/5 ml, granulés pour suspension ...",LINÉZOLIDE,J01XX08
15821,63283095,"ZYVOXID 2 mg/ml, solution pour perfusion",LINÉZOLIDE,J01XX08


In [19]:
mot = "DOLIPRANE"

mask = df_final.astype(str).apply(lambda col: col.str.contains(mot, case=False, na=False))
resultat = df_final[mask.any(axis=1)]

resultat

,cis,nom_commercial,dci,atc
3554,60904643,"CODOLIPRANE 500 mg/30 mg, comprimé",PARACÉTAMOL; PHOSPHATE DE CODÉINE HÉMIHYDRATÉ,
3555,62351650,"CODOLIPRANE 500 mg/30 mg, comprimé effervescen...",PARACÉTAMOL; PHOSPHATE DE CODÉINE HÉMIHYDRATÉ,
3556,62049552,"CODOLIPRANE 500 mg/30 mg, gélule",PARACÉTAMOL; PHOSPHATE DE CODÉINE HÉMIHYDRATÉ,
3557,64406362,"CODOLIPRANE ADULTES 400 mg/20 mg, comprimé séc...",CODÉINE (PHOSPHATE DE) HÉMIHYDRATÉ; PARACÉTAMOL,
4464,64793681,"DOLIPRANE 100 mg, poudre pour solution buvable...",PARACÉTAMOL,
4465,66057393,"DOLIPRANE 100 mg, suppositoire sécable",PARACÉTAMOL,
4466,60234100,"DOLIPRANE 1000 mg, comprimé",PARACÉTAMOL,
4467,66567738,"DOLIPRANE 1000 mg, comprimé effervescent sécable",PARACÉTAMOL,
4468,69309629,"DOLIPRANE 1000 mg, gélule",PARACÉTAMOL,
4469,61681702,"DOLIPRANE 1000 mg, poudre pour solution buvabl...",PARACÉTAMOL,
